## What does Reddit think about the new MBP?
### Is it da bomb, or did it just bomb?

My friends and I have been talking today about Apple's announcement of their new Macbook Pro line.
I personally own an Apple TV, Ipad mini, Macbook Pro, and an iPhone. I was definitely looking forward
to see what Apple was going to come out with the new Macbook Pro line. My thoughts on the announcement aside,
it seemed to me like the overwhelming majority of users on Reddit [didn't come away very impressed](https://www.reddit.com/r/apple/comments/59plnp/lets_talk_about_those_prices/) with the announcements Apple made in regards to the new Macbook Pro. I thought this would be a good opportunity to play with Reddit's API and to try out some rudimentary sentiment analysis.

## Tech

To talk to Reddit, I'm using the aptly named, [**PRAW**](https://praw.readthedocs.io/en/stable/index.html) library. PRAW stands for the Python Reddit Api Wrapper. Nice.

Now for the sentiment analysis, I'm going to use the [**TextBlob**](https://textblob.readthedocs.io/en/dev/) library. TextBlob is a library that provides an easy to use interface for a lot of common natural language processing tasks.

## Get the data

To begin, we instantiate praw's Reddit class with an appropriate user agent. Be careful how you define your user agent. Doing so incorrectly can get you banned according to Reddit. Fortunately, writing a user agent isn't hard at all. You can find the official documentation on how to do so on the [Reddit API wiki page](https://github.com/reddit/reddit/wiki/API). 

As of this writing, the format should look something like this (taken from the aformentioned wiki):

    <platform>:<app ID>:<version string> (by /u/<reddit username>)

Example: 

    User-Agent: android:com.example.myredditapp:v1.2.3 (by /u/kemitche)


In [7]:
import praw

# I saved my user agent string in a local file
# since one should use their own
with open('./.user_agent') as f:
    user_agent = f.read()

# instantiate Reddit connection class
r = praw.Reddit(user_agent=user_agent)

# let's get the current top 10 submissions
# since praw interacts with reddit lazily, we wrap the method
# call in a list
submissions = list(r.get_subreddit('apple').get_hot(limit=10))
[str(s) for s in submissions]

['374 :: Apple Keynote, October 2016 | Post-Event Megathread',
 '46 :: Pre-order Shipping Megathread | Mac Lineup, 2016',
 "4880 :: Let's talk about those prices...",
 "2931 :: I dont mind the lack of standard USB, but you're telling me I can't u...",
 '1673 :: Tim Cook: "We think technology should be available to everyone" and t...',
 '1096 :: (UK) 13" MacBook Pro has gone from £999 to £1449',
 "712 :: What is Apple's definition of a pro?",
 "698 :: Let's talk about the touch bar.",
 '593 :: Gone is lit Apple logo on the back of Macbook Pro',
 '468 :: What are you thoughts on the new MacBook Pros? Being honest I am not i...']

Cool, so we have some data. From looking at the actual subreddit in a browser, we can see the two top submissions are official threads so we'll just skip over them.

<a href="http://imgur.com/8MJXgdg"><img src="http://i.imgur.com/8MJXgdg.png" title="source: imgur.com" /></a>

In [20]:
submissions = submissions[2:]
[str(s) for s in submissions]

["4880 :: Let's talk about those prices...",
 "2931 :: I dont mind the lack of standard USB, but you're telling me I can't u...",
 '1673 :: Tim Cook: "We think technology should be available to everyone" and t...',
 '1096 :: (UK) 13" MacBook Pro has gone from £999 to £1449',
 "712 :: What is Apple's definition of a pro?",
 "698 :: Let's talk about the touch bar.",
 '593 :: Gone is lit Apple logo on the back of Macbook Pro',
 '468 :: What are you thoughts on the new MacBook Pros? Being honest I am not i...']

### Submission 1

We'll start by looking at the comments in the first submission about Apple's new pricing. Can you guess what people think?!

In praw, every Submission has a comments attribute which is iterable. This attribute isn't homogeneous. That is, some of the items will be Comment objects, and there may be a MoreComments class in there as well, so we'll need to handle that.

In [47]:
# grab the first submission
submission = submissions[0]

# the actual text is in the body
# attribute of a Comment
def get_comments(submission, n=20):
    """
    Return a list of comments from a submission.
    
    We can't just use submission.comments, because some
    of those comments may be MoreComments classes and those don't
    have bodies.
    
    n is the number of comments we want to limit ourselves to
    from the submission.
    """
    count = 0
    def barf_comments(iterable=submission.comments):
        """
        This generator barfs out comments.
        
        Some comments seem not to have bodies, so we skip those.
        """
        nonlocal count
        for c in iterable:
            if hasattr(c, 'body') and count < n:
                count += 1
                yield c.body
            else:
                # Sometimes c here will be a Comment
                # class which is not iterable.
                if hasattr(c, '__iter__'):
                    yield from barf_comments(c)
                else:
                    # c was a Comment and did not have a body
                    continue
    return barf_comments()
                
comments = list(get_comments(submission))
list(comments)

['I\'m still trying to figure out what justifies a $200 increase on the 13" ***without*** the Touch Bar...',
 "Probably shouldn't have been surprised, but it hit me like a slippery fish. My jaw dropped.\n\nThey've priced me out, sadly.",
 '$4300 to max out the 15 pro...lol',
 "I was waiting for the new Macbooks to replace my water damaged 2015 Macbook Pro.\n\nNow, I'm off to Apple Store to get my Macbook fixed. These prices are insane.",
 'As an Australian, I am expecting to be even less thrilled when regional pricing is revealed. ',
 '2000 euros here in Germany. I am shocked. Really',
 "Ain't paying that much. May have to stick with my base model 2012 air :(",
 "Even without the TouchBar, it is still $1500. \n$200 over the last year's MacBook Pro. \nThe price increase is just insane. ",
 "As expected, this has made Microsoft's Surface line extremely attractive.",
 '$2399USD for a Quad Core i7, 16GB of 2133Mhz RAM, 256GB SSD, and an AMD 400 mobile series, but not even the 480RX equival

## Sentiment analysis!

So now we have the first twenty comments of the first submission.

We'll combine them into one piece of text and determine the overall sentiment from them.

According to the TextBlob docs, this is how to use their sentiment analysis api and how to interpret it


### The sentiment property returns a namedtuple of the form Sentiment(polarity, subjectivity). The polarity score is a float within the range [-1.0, 1.0]. The subjectivity is a float within the range [0.0, 1.0] where 0.0 is very objective and 1.0 is very subjective.

```
>>> testimonial = TextBlob("Textblob is amazingly simple to use. What great fun!")
>>> testimonial.sentiment
Sentiment(polarity=0.39166666666666666, subjectivity=0.4357142857142857)
>>> testimonial.sentiment.polarity
0.39166666666666666
```

In [48]:
from textblob import TextBlob

comment_blob = TextBlob(''.join(comments))
comment_blob.sentiment

Sentiment(polarity=-0.05465367965367966, subjectivity=0.5949222908239302)

Huh, things aren't looking good so far. Well, let's look at more data.

In [54]:
many_more_comments = []
for submission in submissions:
    many_more_comments.extend(get_comments(submission, 200))

In [55]:
len(many_more_comments)

406